# MLflow end-to-end example

In this example we are going to build a model using `mlflow`, pack and deploy locally using `tempo` (in docker and local kubernetes cluster).

We are are going to use follow the MNIST pytorch example from `mlflow`, check this [link](https://github.com/mlflow/mlflow/tree/master/examples/pytorch/MNIST) for more information.



In this example we will:
  * [Train MNIST Model using mlflow and pytorch](#Train-model)
  * [Create tempo artifacts](#Save-model-environment)
  * [Deploy Locally to Docker](#Deploy-to-Docker)
  * [Deploy Locally to Kubernetes](#Deploy-to-Kubernetes)

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

## Train model

We train MNIST model below:

### Install prerequisites

In [1]:
!pip install mlflow 'torchvision>=0.9.1' torch==1.9.0 pytorch-lightning==1.4.0

In [3]:
!rm -fr /tmp/mlflow

In [4]:
%cd /tmp

/tmp


In [5]:
!git clone https://github.com/mlflow/mlflow.git

Cloning into 'mlflow'...
remote: Enumerating objects: 43208, done.
remote: Counting objects: 100% (1650/1650), done.
remote: Compressing objects: 100% (857/857), done.
remote: Total 43208 (delta 992), reused 1271 (delta 767), pack-reused 41558
Receiving objects: 100% (43208/43208), 38.22 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (33107/33107), done.


### Train model using `mlflow`

In [7]:
%cd mlflow/examples/pytorch/MNIST

/tmp/mlflow/examples/pytorch/MNIST


In [20]:
!mlflow run . --no-conda

2021/09/07 09:54:39 INFO mlflow.projects.utils: === Created directory /tmp/tmp583_9vom for downloading remote URIs passed to arguments of type 'path' ===
2021/09/07 09:54:39 INFO mlflow.projects.backend.local: === Running command 'python mnist_autolog_example.py \
  --max_epochs 5 \
  --gpus 0 \
  --accelerator None \
  --batch_size 64 \
  --num_workers 3 \
  --lr 0.001 \
  --es_patience 3 \
  --es_mode min \
  --es_verbose True \
  --es_monitor val_loss
' in run with ID 'c478ef52c6ec4c2fbaba3ad9f788a101' === 
/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. 

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
Testing: 0it [00:00, ?it/s]/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
Testing:  91%|████████████████████████████▏  | 143/157 [00:00<00:00, 205.75it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_acc': 0.9678543210029602}
--------------------------------------------------------------------

In [21]:
!tree -L 1 mlruns/0

mlruns/0
├── 3243d1faac7c4eb69ee11e2b77766e2a
├── c478ef52c6ec4c2fbaba3ad9f788a101
└── meta.yaml

2 directories, 1 file


### Choose test image

In [10]:
from torchvision import datasets

mnist_test = datasets.MNIST('/tmp/data', train=False, download=True)
# change the index below to get a different image for testing
mnist_test = list(mnist_test)[0]
img, category = mnist_test
display(img)
print(category)

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


7


### Tranform test image to numpy

In [11]:
import numpy as np
img_np = np.asarray(img).reshape((1, 28*28)).astype(np.float32)

## Save model environment

In [22]:
import glob
import os

files = glob.glob("mlruns/0/*/")
files.sort(key=os.path.getmtime)

ARTIFACTS_FOLDER = os.path.join(
    os.getcwd(),
    files[-1],
    "artifacts",
    "model"
)
assert os.path.exists(ARTIFACTS_FOLDER)
print(ARTIFACTS_FOLDER)

/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/c478ef52c6ec4c2fbaba3ad9f788a101/artifacts/model


### Define `tempo` model

In [23]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model

mlflow_tag = "mlflow"

pytorch_mnist_model = Model(
    name="test-pytorch-mnist",
    platform=ModelFramework.MLFlow,
    local_folder=ARTIFACTS_FOLDER,
    # if we deploy to kube, this defines where the model artifacts are stored
    uri="s3://tempo/basic/mnist",
    description="A pytorch MNIST model",
)



Insights Manager not initialised as empty URL provided.


### Save model (environment) using `tempo`

Tempo hides many details required to save the model environment for `mlserver`:
- Add required runtime dependencies
- Create a conda pack `environment.tar.gz`

In [ ]:
from tempo.serve.loader import save
save(pytorch_mnist_model)

Solving environment: ...working... done

_openmp_mutex-4.5    | 22 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... Ran pip subprocess with arguments:
['/home/sa/miniconda3/envs/tempo-74889c69-e5a7-499d-9d10-867b916d697c/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.hfliozx3.requirements.txt']
Pip subprocess output:
  Using cached mlflow-1.20.2-py3-none-any.whl (14.6 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached cffi-1.14.6-cp37-cp37m-manylinux1_x86_64.whl (402 kB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached ipython-7.26.0-py3-none-any.whl (786 kB)
  Using cached pyDeprecate-0.3.1-py3-none-any.whl (10 kB)
  Using cached pytorch_lightning-1.4.0-py3

Packing environment at '/home/sa/miniconda3/envs/tempo-74889c69-e5a7-499d-9d10-867b916d697c' to '/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/c478ef52c6ec4c2fbaba3ad9f788a101/artifacts/model/environment.tar.gz'
[#############################           ] | 73% Completed | 58.6s

## Deploy to Docker

In [ ]:
from tempo import deploy_local
local_deployed_model = deploy_local(pytorch_mnist_model)

In [ ]:
local_prediction = local_deployed_model.predict(img_np)
print(np.nonzero(local_prediction.flatten() == 0))

In [ ]:
local_deployed_model.undeploy()

## Deploy to Kubernetes

### Prerequisites
 
Create a Kind Kubernetes cluster with Minio and Seldon Core installed using Ansible as described [here](https://tempo.readthedocs.io/en/latest/overview/quickstart.html#kubernetes-cluster-with-seldon-core).

In [ ]:
%cd -0

In [ ]:
!kubectl apply -f k8s/rbac -n seldon

### Upload artifacts to minio

In [ ]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [ ]:
from tempo.serve.loader import upload
upload(pytorch_mnist_model)

In [ ]:
from tempo.serve.metadata import SeldonCoreOptions
runtime_options = SeldonCoreOptions(**{
        "remote_options": {
            "namespace": "seldon",
            "authSecretName": "minio-secret"
        }
    })

### Deploy to `kind`

In [ ]:
from tempo import deploy_remote
remote_deployed_model = deploy_remote(pytorch_mnist_model, options=runtime_options)

In [ ]:
remote_prediction = remote_deployed_model.predict(img_np)
print(np.nonzero(remote_prediction.flatten() == 0))

In [ ]:
remote_deployed_model.undeploy()